In [5]:
import polars as pl
import altair as alt

In [6]:
heap_bench_df = pl.read_csv("../data/runtime/bench-heap-feather.csv")

heap_bench_df = heap_bench_df.with_columns(
    pl.col("environment").str.split("-")
).with_columns(
    pl.col("environment").list.get(0),
    pl.col("environment").list.get(1).alias("heap in kb").cast(pl.Int32),
).group_by(["benchmark", "board", "environment", "heap in kb"]) \
.agg(pl.mean("execution_time_us"), pl.std("execution_time_us").alias("execution_time_us_std"))

heap_bench_df

benchmark,board,environment,heap in kb,execution_time_us,execution_time_us_std
str,str,str,i32,f64,f64
"""xgboost""","""adafruit-feather-nrf52840-sens…","""jerryscript""",100,5569284.8,0.447214
"""libud""","""adafruit-feather-nrf52840-sens…","""lua""",55,734603.2,830.43886
"""tarfind""","""adafruit-feather-nrf52840-sens…","""jerryscript""",28,705300.6,759.58956
"""crc_32""","""adafruit-feather-nrf52840-sens…","""jerryscript""",80,5947761.8,0.447214
"""xgboost""","""adafruit-feather-nrf52840-sens…","""jerryscript""",60,5600529.8,0.447214
…,…,…,…,…,…
"""md5""","""adafruit-feather-nrf52840-sens…","""lua""",115,784640.6,1.516575
"""crc_32""","""adafruit-feather-nrf52840-sens…","""micropython""",72,2.675349e6,1.224745
"""md5""","""adafruit-feather-nrf52840-sens…","""jerryscript""",60,5180725.2,0.447214


In [9]:
chart = alt.Chart(heap_bench_df.filter(pl.col("benchmark") != "xgboost")).encode(
    alt.X("heap in kb", title="Heap Size (KB)"),
    alt.Y("execution_time_us", title="Execution Time in µs"),
    alt.YError("execution_time_us_std"),
    alt.Color("benchmark")
)

(chart.mark_line() + chart.mark_errorband() + chart.mark_point()) \
.facet(
    column= alt.Column("environment", title="")
).display(scaleFactor=2)

alt.FacetChart(...)

In [ ]:
chart = alt.Chart(heap_bench_df.filter(pl.col("benchmark") == "xgboost")).encode(
    alt.X("heap in kb", title="Heap Size (KB)"),
    alt.Y("execution_time_us"),
    alt.YError("execution_time_us_std"),
    alt.Color("benchmark")
)

(chart.mark_line() + chart.mark_errorband() + chart.mark_point()) \
.facet(
    column= alt.Column("environment", title="")
).display(scaleFactor=2)

alt.FacetChart(...)